# Motion Prediction with Graph Neural Network

Getting Started with this notebook:
- You might need to add the utility script: kaggle_l5kit
- Also activate the GPU as an accelerator on the right of your screen

# Summary

* optimizer: Adam
* scheduler: CosineAnnealing with start lr of 1e-3 and min_lr of 1e-5, T_max = n_epochs
* Batch_size : 150 (choose as great as possible)
* Could train up to 25 epochs during 9h
* Training module : We used Pytorch Lighning as well as torch_geometric, the geometric deep learning module of Pytorch. 


In [1]:
# running the installation can take a couple of minutes

try:
    import zarr
except ModuleNotFoundError:
    !pip install --use-feature=2020-resolver zarr > /dev/null
try:
    import pytorch_lightning
except ModuleNotFoundError:
    !pip install  --use-feature=2020-resolver pytorch-lightning  > /dev/null
    
try:
    import torch_geometric
except ModuleNotFoundError:
    !pip install torch-geometric

!pip install torch-scatter==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-sparse==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-cluster==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-spline-conv==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-geometric

!pip install neptune-client

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_scatter-latest%2Bcu102-cp36-cp36m-linux_x86_64.whl (11.6 MB)
ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_i

    Found existing installation: torch-cluster 1.5.8
    Uninstalling torch-cluster-1.5.8:
      Successfully uninstalled torch-cluster-1.5.8
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/l5kit/bin/python -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_spline_conv-latest%2Bcu102-cp36-cp36m-linux_x86_64.whl (6.1 MB)
ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map


ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info


ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info


In [2]:
import zarr
from abc import ABC
from pathlib import Path
import matplotlib.pyplot as plt
from numcodecs import blosc
import pandas as pd, numpy as np

import bisect
import itertools as it
from tqdm.notebook import tqdm


import torch
from torch import nn, optim 
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

from pytorch_lightning import Trainer
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import pickle, copy, re, time, datetime, random, warnings, gc

In [3]:
from torch import nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, BatchNorm1d as BN
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import global_max_pool
from torch_geometric.nn import PairNorm

In [4]:
DATA_ROOT = Path("/data/lyft-motion-prediction-autonomous-vehicles")
#DATA_ROOT = Path("../input/lyft-motion-prediction-autonomous-vehicles")
TRAIN_ZARR = "scenes/train.zarr"
VALID_ZARR = "scenes/validate.zarr"

print("DATA_ROOT: {}\TRAIN_ZARR: {}\nVALID_ZARR: {}".format(DATA_ROOT, TRAIN_ZARR, VALID_ZARR))

DATA_ROOT: /data/lyft-motion-prediction-autonomous-vehicles\TRAIN_ZARR: scenes/train.zarr
VALID_ZARR: scenes/validate.zarr


In [5]:
HBACKWARD = 15
HFORWARD = 50
NFRAMES = 10
FRAME_STRIDE = 15
AGENT_FEATURE_DIM = 8
MAX_AGENTS = 150

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

NUM_WORKERS = 4
BATCH_SIZE = 15
EPOCHS=15
GRADIENT_CLIP_VAL = 1.0
LIMIT_VAL_BATCHES = 0.20
ROOT = "GNN"

Path(ROOT).mkdir(exist_ok=True, parents=True)

DEVICE: cuda


In [6]:
TIME_FORMAT = r"%Y-%m-%dT%H:%M:%S%Z"
def get_utc():
    return datetime.datetime.now(datetime.timezone.utc).strftime(TIME_FORMAT)

# Dataset

In [7]:
PERCEPTION_LABELS = [
    "PERCEPTION_LABEL_NOT_SET",
    "PERCEPTION_LABEL_UNKNOWN",
    "PERCEPTION_LABEL_DONTCARE",
    "PERCEPTION_LABEL_CAR",
    "PERCEPTION_LABEL_VAN",
    "PERCEPTION_LABEL_TRAM",
    "PERCEPTION_LABEL_BUS",
    "PERCEPTION_LABEL_TRUCK",
    "PERCEPTION_LABEL_EMERGENCY_VEHICLE",
    "PERCEPTION_LABEL_OTHER_VEHICLE",
    "PERCEPTION_LABEL_BICYCLE",
    "PERCEPTION_LABEL_MOTORCYCLE",
    "PERCEPTION_LABEL_CYCLIST",
    "PERCEPTION_LABEL_MOTORCYCLIST",
    "PERCEPTION_LABEL_PEDESTRIAN",
    "PERCEPTION_LABEL_ANIMAL",
    "AVRESEARCH_LABEL_DONTCARE",
]
KEPT_PERCEPTION_LABELS = [
    "PERCEPTION_LABEL_UNKNOWN",
    "PERCEPTION_LABEL_CAR",
    "PERCEPTION_LABEL_CYCLIST",
    "PERCEPTION_LABEL_PEDESTRIAN",
]
KEPT_PERCEPTION_LABELS_DICT = {label:PERCEPTION_LABELS.index(label) for label in KEPT_PERCEPTION_LABELS}
KEPT_PERCEPTION_KEYS = sorted(KEPT_PERCEPTION_LABELS_DICT.values())

In [8]:
class LabelEncoder:
    def  __init__(self, max_size=500, default_val=-1):
        self.max_size = max_size
        self.labels = {}
        self.default_val = default_val

    @property
    def nlabels(self):
        return len(self.labels)

    def reset(self):
        self.labels = {}

    def partial_fit(self, keys):
        nlabels = self.nlabels
        available = self.max_size - nlabels

        if available < 1:
            return

        keys = set(keys)
        new_keys = list(keys - set(self.labels))

        if not len(new_keys):
            return
        
        self.labels.update(dict(zip(new_keys, range(nlabels, nlabels + available) )))
    
    def fit(self, keys):
        self.reset()
        self.partial_fit(keys)

    def get(self, key):
        return self.labels.get(key, self.default_val)
    
    def transform(self, keys):
        return np.array(list(map(self.get, keys)))

    def fit_transform(self, keys, partial=True):
        self.partial_fit(keys) if partial else self.fit(keys)
        return self.transform(keys)

In [9]:
class CustomLyftDataset(Dataset):
    feature_mins = np.array([-17.336, -27.137, 0. , 0., 0. , -3.142, -37.833, -65.583],
    dtype="float32")[None,None, None]

    feature_maxs = np.array([17.114, 20.787, 42.854, 42.138,  7.079,  3.142, 29.802, 35.722],
    dtype="float32")[None,None, None]



    def __init__(self, zdataset, scenes=None, nframes=10, frame_stride=15, hbackward=10, 
                 hforward=50, max_agents=150, agent_feature_dim=8):
        """
        Custom Lyft dataset reader.
        
        Parmeters:
        ----------
        zdataset: zarr dataset
            The root dataset, containing scenes, frames and agents
            
        nframes: int
            Number of frames per scene
            
        frame_stride: int
            The stride when reading the **nframes** frames from a scene
            
        hbackward: int
            Number of backward frames from  current frame
            
        hforward: int
            Number forward frames from current frame
        
        max_agents: int 
            Max number of agents to read for each target frame. Note that,
            this also include the backward agents but not the forward ones.
        """
        super().__init__()
        self.zdataset = zdataset
        self.scenes = scenes if scenes is not None else []
        self.nframes = nframes
        self.frame_stride = frame_stride
        self.hbackward = hbackward
        self.hforward = hforward
        self.max_agents = max_agents

        self.nread_frames = (nframes-1)*frame_stride + hbackward + hforward

        self.frame_fields = ['timestamp', 'agent_index_interval']

        self.agent_feature_dim = agent_feature_dim

        self.filter_scenes()
      
    def __len__(self):
        return len(self.scenes)

    def filter_scenes(self):
        self.scenes = [scene for scene in self.scenes if self.get_nframes(scene) > self.nread_frames]


    def __getitem__(self, index):
        return self.read_frames(scene=self.scenes[index])

    def get_nframes(self, scene, start=None):
        frame_start = scene["frame_index_interval"][0]
        frame_end = scene["frame_index_interval"][1]
        nframes = (frame_end - frame_start) if start is None else ( frame_end - max(frame_start, start) )
        return nframes


    def _read_frames(self, scene, start=None):
        nframes = self.get_nframes(scene, start=start)
        assert nframes >= self.nread_frames

        frame_start = scene["frame_index_interval"][0]

        start = start or frame_start + np.random.choice(nframes-self.nread_frames)
        frames = self.zdataset.frames.get_basic_selection(
            selection=slice(start, start+self.nread_frames),
            fields=self.frame_fields,
            )
        return frames
    

    def parse_frame(self, frame):
        return frame

    def parse_agent(self, agent):
        return agent

    def read_frames(self, scene, start=None,  white_tracks=None, encoder=False):
        white_tracks = white_tracks or []
        frames = self._read_frames(scene=scene, start=start)

        agent_start = frames[0]["agent_index_interval"][0]
        agent_end = frames[-1]["agent_index_interval"][1]

        agents = self.zdataset.agents[agent_start:agent_end]


        X = np.zeros((self.nframes, self.max_agents, self.hbackward, self.agent_feature_dim), dtype=np.float32)
        target = np.zeros((self.nframes, self.max_agents, self.hforward, 2),  dtype=np.float32)
        target_availability = np.zeros((self.nframes, self.max_agents, self.hforward), dtype=np.uint8)
        X_availability = np.zeros((self.nframes, self.max_agents, self.hbackward), dtype=np.uint8)

        for f in range(self.nframes):
            backward_frame_start = f*self.frame_stride
            forward_frame_start = f*self.frame_stride+self.hbackward
            backward_frames = frames[backward_frame_start:backward_frame_start+self.hbackward]
            forward_frames = frames[forward_frame_start:forward_frame_start+self.hforward]

            backward_agent_start = backward_frames[-1]["agent_index_interval"][0] - agent_start
            backward_agent_end = backward_frames[-1]["agent_index_interval"][1] - agent_start

            backward_agents = agents[backward_agent_start:backward_agent_end]

            le = LabelEncoder(max_size=self.max_agents)
            le.fit(white_tracks)
            le.partial_fit(backward_agents["track_id"])

            for iframe, frame in enumerate(backward_frames):
                backward_agent_start = frame["agent_index_interval"][0] - agent_start
                backward_agent_end = frame["agent_index_interval"][1] - agent_start

                backward_agents = agents[backward_agent_start:backward_agent_end]

                track_ids = le.transform(backward_agents["track_id"])
                mask = (track_ids != le.default_val)
                mask_agents = backward_agents[mask]
                mask_ids = track_ids[mask]
                X[f, mask_ids, iframe, :2] = mask_agents["centroid"]
                X[f, mask_ids, iframe, 2:5] = mask_agents["extent"]
                X[f, mask_ids, iframe, 5] = mask_agents["yaw"]
                X[f, mask_ids, iframe, 6:8] = mask_agents["velocity"]

                X_availability[f, mask_ids, iframe] = 1

            
            for iframe, frame in enumerate(forward_frames):
                forward_agent_start = frame["agent_index_interval"][0] - agent_start
                forward_agent_end = frame["agent_index_interval"][1] - agent_start

                forward_agents = agents[forward_agent_start:forward_agent_end]

                track_ids = le.transform(forward_agents["track_id"])
                mask = track_ids != le.default_val

                target[f, track_ids[mask], iframe] = forward_agents[mask]["centroid"]
                target_availability[f, track_ids[mask], iframe] = 1

        target -= X[:,:,[-1], :2]
        target *= target_availability[:,:,:,None]
        X[:,:,:, :2] -= X[:,:,[-1], :2]
        X *= X_availability[:,:,:,None]
        X -= self.feature_mins
        X /= (self.feature_maxs - self.feature_mins)

        if encoder:
            return X, target, target_availability, le
        return X, target, target_availability

# Loss functions

In [10]:
def collate(x):
    x = map(np.concatenate, zip(*x))
    x = map(torch.from_numpy, x)
    return x

In [18]:
def shapefy( xy_pred, xy, xy_av):
    NDIM = 3
    xy_pred = xy_pred.view(-1, HFORWARD, NDIM, 2)
    xy = xy.view(-1, HFORWARD, 2)[:,:,None]
    xy_av = xy_av.view(-1, HFORWARD)[:,:,None]
    return xy_pred, xy,xy_av

def LyftLoss(c, xy_pred, xy, xy_av):
    c = c.view(-1,c.shape[-1])
    xy_pred, xy, xy_av  = shapefy(xy_pred, xy, xy_av)
    print(xy.shape)
    print(xy_pred.shape)
    
    #pos_x, pos_y = xy.detach().cpu().numpy()[:10, : , 0, :].reshape((10, 50, ))
    
    c = torch.softmax(c, dim=1)
    
    l = torch.sum(torch.mean(torch.square(xy_pred-xy), dim=3)*xy_av, dim=1)
    
    # The LogSumExp trick for better numerical stability
    # https://en.wikipedia.org/wiki/LogSumExp
    m = l.min(dim=1).values
    l = torch.exp(m[:, None]-l)
    
    l = m - torch.log(torch.sum(l*c, dim=1))
    denom = xy_av.max(2).values.max(1).values
    l = torch.sum(l*denom)/denom.sum()
    return 3*l # I found that my loss is usually 3 times smaller than the LB score


def MSE(xy_pred, xy, xy_av):
    xy_pred, xy, xy_av = shapefy(xy_pred, xy, xy_av)
    return 9*torch.mean(torch.sum(torch.mean(torch.square(xy_pred-xy), 3)*xy_av, dim=1))

def MAE(xy_pred, xy, xy_av):
    xy_pred, xy, xy_av = shapefy(xy_pred, xy, xy_av)
    return 9*torch.mean(torch.sum(torch.mean(torch.abs(xy_pred-xy), 3)*xy_av, dim=1))


# Pytorch-Lyghtining Modules

In [19]:
class BaseNet(LightningModule):   
    def __init__(self, batch_size=32, lr=5e-4, weight_decay=1e-8, num_workers=0, 
                 criterion=LyftLoss, data_root=DATA_ROOT,  epochs=1):
        super().__init__()

       
        self.save_hyperparameters(
            dict(
                HBACKWARD = HBACKWARD,
                HFORWARD = HFORWARD,
                NFRAMES = NFRAMES,
                FRAME_STRIDE = FRAME_STRIDE,
                AGENT_FEATURE_DIM = AGENT_FEATURE_DIM,
                MAX_AGENTS = MAX_AGENTS,
                TRAIN_ZARR = TRAIN_ZARR,
                VALID_ZARR = VALID_ZARR,
                batch_size = batch_size,
                lr=lr,
                weight_decay=weight_decay,
                num_workers=num_workers,
                criterion=criterion,
                epochs=epochs,
            )
        )
        
        self._train_data = None
        self._collate_fn = None
        self._train_loader = None

        self.batch_size = batch_size
        self.num_workers = num_workers
        
        
        self.lr = lr
        self.epochs=epochs
        
        self.weight_decay = weight_decay
        self.criterion = criterion
        
        self.data_root = data_root
    

    def train_dataloader(self):
        z = zarr.open(self.data_root.joinpath(TRAIN_ZARR).as_posix(), "r")
        scenes = z.scenes.get_basic_selection(slice(None), fields= ["frame_index_interval"])
        train_data = CustomLyftDataset(
                    z, 
                    scenes = scenes,
                    nframes=NFRAMES,
                    frame_stride=FRAME_STRIDE,
                    hbackward=HBACKWARD,
                    hforward=HFORWARD,
                    max_agents=MAX_AGENTS,
                    agent_feature_dim=AGENT_FEATURE_DIM,
                )
        
        train_loader = DataLoader(train_data, batch_size = self.batch_size,collate_fn=collate,
                                pin_memory=True, num_workers = self.num_workers, shuffle=True)
        self._train_data = train_data
        self._train_loader = train_loader
        
        return train_loader

    def val_dataloader(self):
        z = zarr.open(self.data_root.joinpath(VALID_ZARR).as_posix(), "r")
        scenes = z.scenes.get_basic_selection(slice(None), fields=["frame_index_interval"])
        val_data = CustomLyftDataset(
                    z, 
                    scenes = scenes,
                    nframes=NFRAMES,
                    frame_stride=FRAME_STRIDE,
                    hbackward=HBACKWARD,
                    hforward=HFORWARD,
                    max_agents=MAX_AGENTS,
                    agent_feature_dim=AGENT_FEATURE_DIM,
                )
        
        val_loader = DataLoader(val_data, batch_size = self.batch_size, collate_fn=collate,
                                pin_memory=True, num_workers = self.num_workers, shuffle=True)
        self._val_data = val_data
        self._val_loader = val_loader
        return val_loader

    def validation_epoch_end(self, outputs):
        avg_loss = torch.mean(torch.tensor([x['val_loss'] for x in outputs]))
        avg_mse = torch.mean(torch.tensor([x['val_mse'] for x in outputs]))
        avg_mae = torch.mean(torch.tensor([x['val_mae'] for x in outputs]))
        
        tensorboard_logs = {'val_loss': avg_loss, "val_rmse": torch.sqrt(avg_mse), "val_mae": avg_mae}

        torch.cuda.empty_cache()
        gc.collect()

        return {
            'val_loss': avg_loss,
            'log': tensorboard_logs,
            "progress_bar": {"val_ll": tensorboard_logs["val_loss"], "val_rmse": tensorboard_logs["val_rmse"]}
        }

    
    def configure_optimizers(self):
        optimizer =  optim.Adam(self.parameters(), lr= self.lr, betas= (0.9,0.999), 
                          weight_decay= self.weight_decay, amsgrad=False)
        
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=self.epochs,
            eta_min=1e-5,
        )
        return [optimizer], [scheduler]

# GraphNet

In [20]:
class GraphNet(BaseNet):
    """
    Implement GraphNet model
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        hidden_size = 256
        nb_layers = 7
        nb_input_channels = 120
        nb_output_channels = 300
        self.nb_layers = nb_layers
        self.hidden_size = hidden_size
        self.conv_in_1 = SAGEConv(nb_input_channels, hidden_size)
        self.conv_in_2 = SAGEConv(hidden_size, 2*hidden_size)
        self.conv_in_3 = SAGEConv(2*hidden_size, 2*hidden_size)
        
        self.c_net = nn.Sequential(
            nn.Linear(2*hidden_size, hidden_size), 
            nn.ReLU(),
            nn.Linear(hidden_size, 3)
        )
        
        
        self.conv_out_1 = SAGEConv(2*hidden_size, 2*hidden_size)
        self.conv_out_2 = SAGEConv(2*hidden_size, hidden_size)
        self.conv_out_3 = SAGEConv(hidden_size, hidden_size)
        self.conv_out_4 = SAGEConv(hidden_size, nb_output_channels)
        self.relu = nn.ReLU(inplace=True)
        

    def forward(self, x):

        bsize, npoints, hb, nf = x.shape 

        # Merge time with features
        x = x.reshape(bsize, npoints, hb*nf)
        
        L1, L2 = [], []
        for i in range(MAX_AGENTS):
            for j in range(MAX_AGENTS):
                L1.append(i)
                L2.append(j)
                
        edge_index = torch.tensor([L1, L2], dtype=torch.long).cuda()
        
        out = self.relu(self.conv_in_1(x, edge_index))
        out = self.relu(self.conv_in_2(out, edge_index))
        out = self.relu(self.conv_in_3(out, edge_index))
        c = self.c_net(out)

        out = self.relu(self.conv_out_1(out, edge_index))
        out = self.relu(self.conv_out_2(out, edge_index))
        out = self.relu(self.conv_out_3(out, edge_index))
        out = self.conv_out_4(out, edge_index)
        return c, out

    def get_nb_trainable_params(self):
        """
        Return the number of trainable parameters
        """
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        return sum([np.prod(p.size()) for p in model_parameters])

    def describe(self):
        s = f"nb_layers: {self.nb_layers}; hidden size: {self.hidden_size}"
        return s
    
    def training_step(self, batch, batch_idx):
        x, y, y_av = [b.to(DEVICE) for b in batch]
        c, preds = self(x)
        loss = self.criterion(c,preds,y, y_av)
        
        with torch.no_grad():
            logs = {
                'loss': loss,
                "mse": MSE(preds, y, y_av),
                "mae": MAE(preds, y, y_av),
            }
        return {'loss': loss, 'log': logs, "progress_bar": {"rmse":torch.sqrt(logs["mse"]) }}
    
    @torch.no_grad()
    def validation_step(self, batch, batch_idx):
        x, y, y_av =  [b.to(DEVICE) for b in batch]
        c,preds = self(x)
        loss = self.criterion(c, preds, y, y_av)
        
        val_logs = {
            'val_loss': loss,
            "val_mse": MSE(preds, y, y_av),
            "val_mae": MAE(preds, y, y_av),
        }
        
        return val_logs
    
    

# Training

In [21]:
learning_rate = 5e-4
weight_decay = 5e-7 

In [22]:
torch.backends.cudnn.benchmark =  True

model = GraphNet(batch_size=BATCH_SIZE, lr= learning_rate, weight_decay=weight_decay, num_workers=NUM_WORKERS)

In [23]:
# Load trained model
model.load_state_dict(torch.load('GraphNet.pt'))
model.cuda()

GraphNet(
  (conv_in_1): SAGEConv(120, 256)
  (conv_in_2): SAGEConv(256, 512)
  (conv_in_3): SAGEConv(512, 512)
  (c_net): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=3, bias=True)
  )
  (conv_out_1): SAGEConv(512, 512)
  (conv_out_2): SAGEConv(512, 256)
  (conv_out_3): SAGEConv(256, 256)
  (conv_out_4): SAGEConv(256, 300)
  (relu): ReLU(inplace=True)
)

In [18]:
from pytorch_lightning.loggers.neptune import NeptuneLogger

neptune_logger = NeptuneLogger(
                api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMDk5YjVmYzYtNTU0My00MzhkLWJiYTAtMGM4ZGVhZmEyMTZiIn0=",
                project_name='hvergnes/KaggleGNN',
                params={'epoch_nr': EPOCHS, 'learning_rate': learning_rate, 'batch_size': BATCH_SIZE},  # your hyperparameters, immutable
                tags=['GNN'],  # tags
                )

https://ui.neptune.ai/hvergnes/KaggleGNN/e/KAG2-15


NeptuneLogger will work in online mode


In [ ]:
trainer = Trainer(
    max_epochs=EPOCHS,
    logger=neptune_logger,
    gradient_clip_val=GRADIENT_CLIP_VAL,
    limit_val_batches=LIMIT_VAL_BATCHES,
    gpus=1
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | conv_in_1  | SAGEConv   | 61 K  
1 | conv_in_2  | SAGEConv   | 262 K 
2 | conv_in_3  | SAGEConv   | 524 K 
3 | c_net      | Sequential | 132 K 
4 | conv_out_1 | SAGEConv   | 524 K 
5 | conv_out_2 | SAGEConv   | 262 K 
6 | conv_out_3 | SAGEConv   | 131 K 
7 | conv_out_4 | SAGEConv   | 153 K 
8 | relu       | ReLU       | 0     
/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/home/ubuntu/anaconda3/envs/l5kit/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) insid

## Special Mention

This work was originaly inspired by the different kernels of KKiller during this competition.
We used some of his code as a building block for our model. Please go upvote his kernel here:
https://www.kaggle.com/kneroma/training-motion-prediction-with-pointnet

<div style="text-align: center"><strong>Thanks</strong></div>